In [1]:
import numpy as np


def f(x):
    return np.sin(x) + x**3 + x**2 + 10 * x + 1 + np.cos(5 * x)


# 1. Setup Data
np.random.seed(0)
N_train = 100
X = np.random.uniform(-10, 5, (N_train, 1))
y = f(X).flatten()
print("Number of training points:", N_train)


# Kernel Function (RBF)
def kernel(x1, x2, l=0.5):
    sq_dist = np.sum(x1**2, 1).reshape(-1, 1) + np.sum(x2**2, 1) - 2 * np.dot(x1, x2.T)
    return np.exp(-0.5 / (l**2) * sq_dist)


# 2. Training (Building K and alpha)
K = kernel(X, X)

# FIX: Add "Jitter" / Noise variance to the diagonal
# This makes the matrix invertible
sigma_n = 1e-5
K[np.diag_indices_from(K)] += sigma_n
alpha = np.linalg.solve(K, y)  # K^-1 * y

# 3. Inference (N Test Points)
# We define x_star with shape (N_test, D)
x_star = np.array([1, 2, 3, 4, 5, 6]).reshape(-1, 1)
k_star = kernel(X, x_star)  # Similarity matrix: Shape (N_train, N_test)

# --- PREDICTION (Mean) ---
# Shape: (N_test, N_train) @ (N_train,) -> (N_test,)
mu_star = np.dot(k_star.T, alpha)

# --- UNCERTAINTY (Variance) ---
# 1. Prior Covariance (K_ss) between test points
K_ss = kernel(x_star, x_star)  # Shape (N_test, N_test)

# 2. Posterior Covariance: K_ss - K_*X * K_XX^-1 * K_X*
# This results in a full covariance matrix (N_test, N_test)
covariance_matrix = K_ss - np.dot(k_star.T, np.linalg.solve(K, k_star))

# We only need the diagonal elements (variance at each point)
uncertainty_variance = np.diag(covariance_matrix)
uncertainty_std = np.sqrt(np.maximum(0, uncertainty_variance))

# --- PRINTING ---
true_values = f(x_star).flatten()

print("\n--- Results ---")
print(
    f"{'Input':<10} {'True':<10} {'Pred':<10} {'RelError':<10} {'Std (Unc)':<10} {'95% CI'}"
)
for i in range(len(x_star)):
    x_val = x_star[i, 0]
    mu = mu_star[i]
    std = uncertainty_std[i]
    tv = true_values[i]
    rel_error = np.abs(tv - mu) / (np.abs(tv) + 1e-8)

    # 95% Confidence Interval is roughly +/- 1.96 * std
    ci_lower = mu - 1.96 * std
    ci_upper = mu + 1.96 * std

    print(
        f"{x_val:<10.1f} {tv:<10.4f} {mu:<10.4f} {rel_error:<10.1e} {std:<10.4f} [{ci_lower:.2f}, {ci_upper:.2f}]"
    )

Number of training points: 100

--- Results ---
Input      True       Pred       RelError   Std (Unc)  95% CI
1.0        14.1251    14.1285    2.4e-04    0.0038     [14.12, 14.14]
2.0        33.0702    33.0731    8.7e-05    0.0028     [33.07, 33.08]
3.0        66.3814    66.4215    6.0e-04    0.0041     [66.41, 66.43]
4.0        120.6513   120.5740   6.4e-04    0.0027     [120.57, 120.58]
5.0        201.0323   196.6150   2.2e-02    0.0415     [196.53, 196.70]
6.0        312.8748   38.7002    8.8e-01    0.9655     [36.81, 40.59]
